In [1]:
# general imports
import numpy as np
import time
import os
import math
import matplotlib.pyplot as plt

# torch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.utils.data import ImageFolder, DataLoader

# tensorboardX
from tensorboardX import SummaryWriter
from format import print_iter, summary

In [ ]:
# TODO: Download scripts and data

from format import print_iter

In [ ]:
image_size = (224, 224)
batch_size = 16
num_workers = 4

dataset = ImageFolder("data/images", transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
dataloader = Dataloader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


In [ ]:
def get_padding(output_dim, input_dim, kernel_size, stride):
    """
    Calculates padding given in output and input dim, and parameters of the convolutional layer

    Arguments should all be integers. Use this function to calculate padding for 1 dimesion at a time.
    Output dimensions should be the same or bigger than input dimensions

    Returns 0 if invalid arguments were passed, otherwise returns an int or tuple that represents the padding.
    """

    padding = (((output_dim - 1) * stride) - input_dim + kernel_size) // 2

    if padding < 0:
        return 0
    else:
        return padding

def gen_block(input_channels, output_channels, kernel_size, stride, padding):
    layers = []
    layers += [nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride=stride, padding=padding, bias=False)]
    layers += [nn.BatchNorm2d(output_channels)]
    layers += [nn.ReLU(inplace=True)]
    
    return layers
    
class Generator(nn.Module):
    def __init__(self, channels=3, input_size=100, output_dim=64):
        super(Generator, self).__init__()
        self.channels = channels
        self.input_size = input_size
        self.output_size = output_size
        self.layers = build_layers()
        
    def forward(self, x):
        return self.layers(x).squeeze()
    
    def build_layers(self):
        layers = []
        in_c = self.input_size
        out_c = self.output_size * 8
        
        # dim: out_c x 4 x 4
        layers += gen_block(in_c, out_c, 4, 1, 0)
        in_c = out_c
        out_c = self.output_size * 4
        
        # dim: out_c x 8 x 8
        layers += gen_block(in_c, out_c, 4, 2, 1)
        in_c = out_c
        out_c = self.output_size * 2
        
        # dim: out_c x 16 x 16
        layers += gen_block(in_c, out_c, 4, 2, 1)
        in_c = out_c
        out_c = self.output_size
        
        # dim: out_c x 32 x 32
        layers += gen_block(in_c, out_c, 4, 2, 1)
        in_c = out_c
        out_c = self.channels
        
        # dim: out_c x 64 x 64
        layers += [nn.ConvTranspose2d(in_c, out_c, 4, 2, 1), nn.Tanh()]
        
        return nn.Sequential(*layers)

In [2]:
def discrim_block(input_channels, output_channels, kernel_size, stride, padding):
    layers = []
    layers += [nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride, padding=padding, bias=False)]
    layers += [nn.BatchNorm2d(output_channels)]
    layers += [nn.LeakyReLU(0.2, inplace=True)]
    
    return layers

def Discriminator(nn.Module):
    def __init__(self, channels=3, input_dim=64):
        super(Discriminator, self).__init__()
        self.channels = channels
        self.input_dim = input_dim
        self.layers = build_layers()
        
    def forward(self, x):
        return self.layers(x).squeeze()
    
    def build_layers(self):
        layers = []
        in_c = self.channels
        out_c = self.input_dim
        
        # dim: out_c x 32 x 32
        layers += discrim_block(in_c, out_c, 4, 1, 0)
        in_c = out_c
        out_c = self.input_dim * 2
        
        # dim: out_c x 16 x 16
        layers += discrim_block(in_c, out_c, 4, 2, 1)
        in_c = out_c
        out_c = self.input_dim * 4
        
        # dim: out_c x 8 x 8
        layers += discrim_block(in_c, out_c, 4, 2, 1)
        in_c = out_c
        out_c = self.input_dim * 8
        
        # dim: out_c x 4 x 4
        layers += discrim_block(in_c, out_c, 4, 2, 1)
        in_c = out_c
        out_c = self.channels
        
        # dim: out_c x 64 x 64
        layers += [nn.Conv2d(in_c, 1, 4, 1, 0), nn.Sigmoid()]
        
        return nn.Sequential(*layers)
        

SyntaxError: invalid syntax (<ipython-input-2-476a8b259e60>, line 9)

In [ ]:
gen_input = 100
gen_output = 64

gen = Generator(input_size=gen_input, output_dim=gen_output)
discrim = Discriminator()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device: {}".format(device))
gen.to(device)
discrim.to(device)

learn_rate = 0.001

optG = optim.Adam(gen.parameters(), lr=learn_rate)
optD = optim.Adam(discrim.parameters(), lr=learn_rate)

criterion = nn.BCELoss()

fixed_noise = torch.randn(gen_output, gen_input, 1, 1, device=device)

real_label = 0.9
fake_label = 0.1